### Set up

Importing the data is taken care by kaggle.<br>
Just upload your kaggle.json API token file(You can get it from your accounts page on Kaggle)

In [1]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"sharifahrodziahhanim","key":"64267733de5fb9f88a4aeede1c343330"}'}

In [2]:
!pip3 install geopandas
!pip install cartopy
!pip3 install mplleaflet
! pip install -q kaggle
! pip install torch-geometric


In [3]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import cm, colors

import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.neighbors import kneighbors_graph
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.inspection import permutation_importance
import cartopy.crs as ccrs
import cartopy.feature as cfeature


In [4]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets list

ref                                                               title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
adilshamim8/student-depression-dataset                            Student Depression Dataset                             467020  2025-03-13 03:12:30.423000          18464        292  1.0              
zahidmughal2343/amazon-sales-2025                                 Amazon Sales 2025                                        3617  2025-04-03 22:08:13.607000           3831         47  1.0              
lucass0s0/polycystic-ovary-syndrome-pcos                          Polycystic Ovary Syndrome PCOS                          19089  2025-04-14 12:14:58.850000            548         23  1.0          

In [8]:
!kaggle datasets download -d tsiaras/uk-road-safety-accidents-and-vehicles

Dataset URL: https://www.kaggle.com/datasets/tsiaras/uk-road-safety-accidents-and-vehicles
License(s): DbCL-1.0
uk-road-safety-accidents-and-vehicles.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
! mkdir train
! unzip uk-road-safety-accidents-and-vehicles.zip -d train

mkdir: cannot create directory ‘train’: File exists
Archive:  uk-road-safety-accidents-and-vehicles.zip
replace train/Accident_Information.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train/Accident_Information.csv  
  inflating: train/Vehicle_Information.csv  


In [10]:
for dirname, _, filenames in os.walk('train'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


train/Accident_Information.csv
train/Vehicle_Information.csv


In [11]:
accidents=pd.read_csv("train/Accident_Information.csv")

<ipython-input-11-0ace3edffbf7>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents=pd.read_csv("train/Accident_Information.csv")


## Feature engineering & data preprocessing

In [12]:
# Imports
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import geopandas as gpd
import folium
from folium.plugins import HeatMap

# Load Data with explicit dtype to avoid DtypeWarning
accidents = pd.read_csv(
    "train/Accident_Information.csv",
    dtype={'Accident_Index': str},
    low_memory=False
)

# Feature Engineering & Preprocessing
features = [
    'Latitude', 'Longitude', 'Speed_limit', 'Weather_Conditions',
    'Road_Type', 'Junction_Detail', 'Day_of_Week', 'Number_of_Vehicles'
]
target = 'Accident_Severity'

# Select relevant columns
accidents = accidents[features + [target]].copy()

# Debug: Check missing values
print("Missing values before preprocessing:")
print(accidents.isna().sum())

# Handle missing values (avoid inplace operations)
accidents['Speed_limit'] = accidents['Speed_limit'].fillna(accidents['Speed_limit'].median())
accidents['Weather_Conditions'] = accidents['Weather_Conditions'].fillna('Fine no high winds')
accidents['Road_Type'] = accidents['Road_Type'].fillna('Single carriageway')
accidents['Junction_Detail'] = accidents['Junction_Detail'].fillna('Not at junction or within 20 metres')
accidents['Day_of_Week'] = accidents['Day_of_Week'].fillna(accidents['Day_of_Week'].mode()[0])
accidents['Number_of_Vehicles'] = accidents['Number_of_Vehicles'].fillna(accidents['Number_of_Vehicles'].median())
accidents['Latitude'] = accidents['Latitude'].fillna(accidents['Latitude'].median())
accidents['Longitude'] = accidents['Longitude'].fillna(accidents['Longitude'].median())

# Drop rows with missing target
accidents = accidents.dropna(subset=[target]).reset_index(drop=True)

# Debug: Verify no missing values remain
print("\nMissing values after preprocessing:")
print(accidents.isna().sum())
print(f"\nNumber of samples after preprocessing: {len(accidents)}")

# Debug: Check class distribution
print("\nClass distribution of Accident_Severity:")
print(accidents[target].value_counts())

# Encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_features = ['Weather_Conditions', 'Road_Type', 'Junction_Detail', 'Day_of_Week']
cat_encoded = encoder.fit_transform(accidents[cat_features])
cat_cols = encoder.get_feature_names_out(cat_features)
cat_df = pd.DataFrame(cat_encoded, columns=cat_cols, index=accidents.index)

# Scale numerical features
scaler = StandardScaler()
num_features = ['Latitude', 'Longitude', 'Speed_limit', 'Number_of_Vehicles']
num_scaled = scaler.fit_transform(accidents[num_features])
num_df = pd.DataFrame(num_scaled, columns=num_features, index=accidents.index)

# Combine features
X = pd.concat([num_df, cat_df], axis=1)
y = LabelEncoder().fit_transform(accidents[target])

# Debug: Verify X has no NaNs
print("\nChecking for NaNs in X:")
print(X.isna().sum().sum(), "missing values in X")

# Convert to tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)


Missing values before preprocessing:
Latitude              174
Longitude             175
Speed_limit            37
Weather_Conditions      0
Road_Type               0
Junction_Detail         0
Day_of_Week             0
Number_of_Vehicles      0
Accident_Severity       0
dtype: int64

Missing values after preprocessing:
Latitude              0
Longitude             0
Speed_limit           0
Weather_Conditions    0
Road_Type             0
Junction_Detail       0
Day_of_Week           0
Number_of_Vehicles    0
Accident_Severity     0
dtype: int64

Number of samples after preprocessing: 2047256

Class distribution of Accident_Severity:
Accident_Severity
Slight     1734548
Serious     286339
Fatal        26369
Name: count, dtype: int64

Checking for NaNs in X:
0 missing values in X


## Train-Test Split

In [13]:
# Train-Validation-Test Split
train_idx, temp_idx, y_train, y_temp = train_test_split(
    np.arange(X_tensor.size(0)), y_tensor.numpy(), test_size=0.3, random_state=42, stratify=y_tensor.numpy()
)
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, random_state=42, stratify=y_temp
)

train_mask = torch.zeros(X_tensor.size(0), dtype=torch.bool)
val_mask = torch.zeros(X_tensor.size(0), dtype=torch.bool)
test_mask = torch.zeros(X_tensor.size(0), dtype=torch.bool)
train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

## Graph Construction

In [14]:
# Create Edges using Spatial Nearest Neighbors (KNN Graph)
n_neighbors = 5
adj_matrix = kneighbors_graph(X[['Latitude', 'Longitude']], n_neighbors, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)


# GNN model

In [15]:
# GNN Model
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim // 2)
        self.conv3 = GCNConv(hidden_dim // 2, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return x

# Initialize Model
input_dim = X.shape[1]
hidden_dim = 64
output_dim = len(np.unique(y))
model = GNNModel(input_dim, hidden_dim, output_dim)

# Class Weights
classes = np.unique(y)
cw = compute_class_weight('balanced', classes=classes, y=y)
class_weights = torch.tensor(cw, dtype=torch.float32)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

## Training

In [16]:
# Training with Early Stopping
def train_model(model, X, y, edge_index, train_mask, val_mask, epochs=10, patience=10):
    best_val_loss = float('inf')
    patience_counter = 0
    model.train()

    # Lists to store metrics
    train_losses = []
    val_losses = []
    val_accuracies = []

    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(X, edge_index)
        loss = criterion(out[train_mask], y[train_mask])
        loss.backward()
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_out = model(X, edge_index)
            val_loss = criterion(val_out[val_mask], y[val_mask])
            val_acc = (val_out[val_mask].argmax(dim=1) == y[val_mask]).float().mean().item()
        model.train()

        # Store metrics
        train_losses.append(loss.item())
        val_losses.append(val_loss.item())
        val_accuracies.append(val_acc)

        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

    return train_losses, val_losses, val_accuracies

## Evaluation

In [17]:
# Evaluation
def evaluate(model, X, y, edge_index, mask, set_name="Test"):
    model.eval()
    with torch.no_grad():
        out = model(X, edge_index)
        preds = out[mask].argmax(dim=1)
        actuals = y[mask]
        acc = (preds == actuals).float().mean().item()
        print(f"\n{set_name} Accuracy: {acc:.4f}")
        print(classification_report(actuals.cpu(), preds.cpu(), zero_division=0))
    return preds.cpu().numpy(), actuals.cpu().numpy()

# Train and Evaluate
train_losses, val_losses, val_accuracies = train_model(model, X_tensor, y_tensor, edge_index, train_mask, val_mask)
test_preds, test_actuals = evaluate(model, X_tensor, y_tensor, edge_index, test_mask, "Test")

Epoch 1/10, Train Loss: 1.1790, Val Loss: 1.1582, Val Acc: 0.0867
Epoch 2/10, Train Loss: 1.1644, Val Loss: 1.1455, Val Acc: 0.0847
Epoch 3/10, Train Loss: 1.1510, Val Loss: 1.1343, Val Acc: 0.0826
Epoch 4/10, Train Loss: 1.1390, Val Loss: 1.1245, Val Acc: 0.0798
Epoch 5/10, Train Loss: 1.1299, Val Loss: 1.1161, Val Acc: 0.0762
Epoch 6/10, Train Loss: 1.1207, Val Loss: 1.1090, Val Acc: 0.0730
Epoch 7/10, Train Loss: 1.1134, Val Loss: 1.1031, Val Acc: 0.0759
Epoch 8/10, Train Loss: 1.1072, Val Loss: 1.0982, Val Acc: 0.0995
Epoch 9/10, Train Loss: 1.1024, Val Loss: 1.0943, Val Acc: 0.1510
Epoch 10/10, Train Loss: 1.0979, Val Loss: 1.0911, Val Acc: 0.2114

Test Accuracy: 0.2125
              precision    recall  f1-score   support

           0       0.02      0.61      0.03      3956
           1       0.13      0.35      0.19     42951
           2       0.87      0.18      0.30    260182

    accuracy                           0.21    307089
   macro avg       0.34      0.38      0.18 

## Visualization

In [18]:
# Set Seaborn style for better aesthetics
sns.set(style="whitegrid", font_scale=1.2)

# 1. Class Distribution Bar Plot
plt.figure(figsize=(8, 6))
class_counts = accidents['Accident_Severity'].value_counts()
sns.barplot(x=class_counts.index, y=class_counts.values, palette='viridis')
plt.title('Distribution of Accident Severity Classes', fontsize=14)
plt.xlabel('Accident Severity', fontsize=12)
plt.ylabel('Number of Accidents', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('class_distribution.png', dpi=300)
plt.close()
print("Class distribution plot saved as 'class_distribution.png'")

<ipython-input-18-f97c79445d5d>:7: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=class_counts.index, y=class_counts.values, palette='viridis')


Class distribution plot saved as 'class_distribution.png'


In [19]:
# 2. Confusion Matrix Heatmap
cm = confusion_matrix(test_actuals, test_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=np.unique(accidents['Accident_Severity']),
            yticklabels=np.unique(accidents['Accident_Severity']))
plt.title('Confusion Matrix for Test Set', fontsize=14)
plt.xlabel('Predicted Severity', fontsize=12)
plt.ylabel('Actual Severity', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300)
plt.close()
print("Confusion matrix heatmap saved as 'confusion_matrix.png'")

Confusion matrix heatmap saved as 'confusion_matrix.png'


In [22]:
# 5. Training and Validation Loss/Accuracy Curves
fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12, color='tab:blue')
ax1.plot(train_losses, label='Train Loss', color='tab:blue')
ax1.plot(val_losses, label='Validation Loss', color='tab:cyan')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
ax2.set_ylabel('Validation Accuracy', fontsize=12, color='tab:red')
ax2.plot(val_accuracies, label='Validation Accuracy', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
ax2.legend(loc='upper right')

plt.title('Training and Validation Metrics', fontsize=14)
fig.tight_layout()
plt.savefig('training_curves.png', dpi=300)
plt.close()
print("Training curves plot saved as 'training_curves.png'")

Training curves plot saved as 'training_curves.png'


In [21]:
# Visualization (Heatmap with Folium)
model.eval()
with torch.no_grad():
    pred_labels = model(X_tensor, edge_index).argmax(dim=1).numpy()

# Create a heatmap of predicted hotspots
gdf = gpd.GeoDataFrame(
    accidents,
    geometry=gpd.points_from_xy(accidents.Longitude, accidents.Latitude),
    crs='EPSG:4326'
)
gdf['pred'] = pred_labels

# Focus on severe accidents (e.g., class 0 or 1)
severe = gdf[gdf['pred'].isin([0, 1])]
m = folium.Map(location=[gdf['Latitude'].mean(), gdf['Longitude'].mean()], zoom_start=6)
heat_data = [[row['Latitude'], row['Longitude']] for _, row in severe.iterrows()]
HeatMap(heat_data, radius=10).add_to(m)
m.save("accident_hotspots.html")
print("Heatmap saved as 'accident_hotspots.html'")

Heatmap saved as 'accident_hotspots.html'
